In [1]:
from keras.datasets import fashion_mnist
import numpy as np
from tensorflow.keras.utils import to_categorical
import wandb
from sklearn.metrics import accuracy_score

In [2]:
# Loading the data
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Normalize the data
x_train = x_train.reshape(-1, 784) / 255.0
x_test = x_test.reshape(-1, 784) / 255.0

# One-hot encode labels
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

# Split validation data
val_split = int(0.1 * x_train.shape[0])

# Create validation data
x_val, y_val = x_train[:val_split], y_train[:val_split]
x_train, y_train = x_train[val_split:], y_train[val_split:]

x_train = x_train.T
y_train = y_train.T
x_val = x_val.T
y_val = y_val.T
x_test = x_test.T
y_test = y_test.T

x_train.shape, y_train.shape

((784, 54000), (10, 54000))

In [3]:
class FeedforwardNeuralNetwork:
    def __init__(self, input_size, hidden_layers, output_size):
        """
        Initialize the neural network.
        :param input_size: Number of input features.
        :param hidden_layers: List containing the number of neurons in each hidden layer.
        :param output_size: Number of output classes.
        """
        self.layers = []
        self.weights = []
        self.biases = []
        self.velocities_w = []  # For momentum-based optimizers
        self.velocities_b = []
        self.squared_w = []  # For RMSprop/Adam
        self.squared_b = []
        self.moments_w = []  # For Nadam
        self.moments_b = []

        # Input layer to first hidden layer
        prev_size = input_size
        for layer_size in hidden_layers:
            self.weights.append(np.random.randn(layer_size, prev_size))
            self.biases.append(np.zeros((layer_size, 1)))
            self.velocities_w.append(np.zeros((layer_size, prev_size)))
            self.velocities_b.append(np.zeros((layer_size, 1)))
            self.squared_w.append(np.zeros((layer_size, prev_size)))
            self.squared_b.append(np.zeros((layer_size, 1)))
            self.moments_w.append(np.zeros((layer_size, prev_size)))
            self.moments_b.append(np.zeros((layer_size, 1)))
            prev_size = layer_size

        # Hidden layers to output layer
        self.weights.append(np.random.randn(output_size, prev_size))
        self.biases.append(np.zeros((output_size, 1)))
        self.velocities_w.append(np.zeros((output_size, prev_size)))
        self.velocities_b.append(np.zeros((output_size, 1)))
        self.squared_w.append(np.zeros((output_size, prev_size)))
        self.squared_b.append(np.zeros((output_size, 1)))
        self.moments_w.append(np.zeros((output_size, prev_size)))
        self.moments_b.append(np.zeros((output_size, 1)))
    
    def train(self, x_train, y_train, x_val, y_val, epochs, learning_rate, optimizer='sgd', batch_size=32):
        for epoch in range(epochs):
            indices = np.arange(x_train.shape[1])
            np.random.shuffle(indices)
            x_train, y_train = x_train[:,indices], y_train[:,indices]
            
            for i in range(0, x_train.shape[1], batch_size):
                x_batch = x_train[:,i:i+batch_size]
                y_batch = y_train[:,i:i+batch_size]
                self.forward(x_batch)
                self.backpropagation(x_batch, y_batch, learning_rate, optimizer)
        
            train_pred, val_pred = self.forward(x_train), self.forward(x_val)
            train_loss, val_loss = self.cross_entropy(train_pred, y_train), self.cross_entropy(val_pred, y_val)
            train_cls, val_cls = np.argmax(train_pred, axis = 0), np.argmax(val_pred, axis = 0)
            train_acc, val_acc = accuracy_score(np.argmax(y_train, axis=0), train_cls), accuracy_score(np.argmax(y_val, axis=0), val_cls)

            print(f"Epoch {epoch}: Training Loss = {train_loss:.4f} Validation Loss = {val_loss:.4f}")
            print(f"Training Accuracy = {train_acc:.4f} Validation Accuracy = {val_acc:.4f}")

            wandb.log({
                "epoch" : epoch, 
                "train_loss" : train_loss, 
                "val_loss" : val_loss, 
                "train_acc" : train_acc, 
                "val_acc" : val_acc
            })
        
        wandb.log({"Validation Loss" : val_loss})
        
    
    @staticmethod
    def cross_entropy(y_pred, y_true) :
        eps = 1e-8 
        y_pred = np.clip(y_pred, eps, 1 - eps)
        return -np.sum(y_true * np.log(y_pred)) / y_true.shape[1]

    def sigmoid(self, x):
        """Sigmoid activation function."""
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        """Derivative of the sigmoid function."""
        return x * (1 - x)

    def softmax(self, x):
        """Softmax activation function."""
        exp_x = np.exp(x - np.max(x, axis=0, keepdims=True))
        return exp_x / np.sum(exp_x, axis=0, keepdims=True)

    def forward(self, x):
        """
        Perform a forward pass through the network.
        :param x: Input data.
        :return: Output probabilities.
        """
        self.layers = [x]
        for w, b in zip(self.weights[:-1], self.biases[:-1]):
            x = self.sigmoid(np.dot(w,x) + b)
            # x = self.sigmoid(np.dot(x, w) + b)
            self.layers.append(x)

        # Output layer with softmax
        output = self.softmax(np.dot(self.weights[-1], x) + self.biases[-1])
        self.layers.append(output)
        return output

    def backpropagation(self, x, y, learning_rate, optimizer='sgd', beta1=0.9, beta2=0.999, epsilon=1e-8):
        m = y.shape[1]  # Number of samples
        deltas = [self.layers[-1] - y]  # Output layer error

        # Backpropagate errors for hidden layers
        for i in range(len(self.weights) - 1, 0, -1):
            deltas.append(self.weights[i].T.dot(deltas[-1]) * self.sigmoid_derivative(self.layers[i]))
        deltas.reverse()

        # Update weights and biases based on optimizer
        for i in range(len(self.weights)):
            dw = (self.layers[i].dot(deltas[i].T)).T / m
            db = np.mean(deltas[i], axis=1, keepdims=True)

            if optimizer == 'sgd':
                self.weights[i] -= learning_rate * dw
                self.biases[i] -= learning_rate * db
            
            elif optimizer == 'momentum':
                self.velocities_w[i] = beta1 * self.velocities_w[i] - learning_rate * dw
                self.velocities_b[i] = beta1 * self.velocities_b[i] - learning_rate * db
                self.weights[i] += self.velocities_w[i]
                self.biases[i] += self.velocities_b[i]
            
            elif optimizer == 'nag':
                prev_w = self.velocities_w[i]
                prev_b = self.velocities_b[i]
                self.velocities_w[i] = beta1 * prev_w - learning_rate * dw
                self.velocities_b[i] = beta1 * prev_b - learning_rate * db
                self.weights[i] += -beta1 * prev_w + (1 + beta1) * self.velocities_w[i]
                self.biases[i] += -beta1 * prev_b + (1 + beta1) * self.velocities_b[i]
            
            elif optimizer == 'rmsprop':
                self.squared_w[i] = beta2 * self.squared_w[i] + (1 - beta2) * (dw ** 2)
                self.squared_b[i] = beta2 * self.squared_b[i] + (1 - beta2) * (db ** 2)
                self.weights[i] -= learning_rate * dw / (np.sqrt(self.squared_w[i]) + epsilon)
                self.biases[i] -= learning_rate * db / (np.sqrt(self.squared_b[i]) + epsilon)
            
            elif optimizer == 'adam' or optimizer == 'nadam':
                self.velocities_w[i] = beta1 * self.velocities_w[i] + (1 - beta1) * dw
                self.velocities_b[i] = beta1 * self.velocities_b[i] + (1 - beta1) * db
                self.squared_w[i] = beta2 * self.squared_w[i] + (1 - beta2) * (dw ** 2)
                self.squared_b[i] = beta2 * self.squared_b[i] + (1 - beta2) * (db ** 2)
                if optimizer == 'nadam':
                    dw = beta1 * self.velocities_w[i] + (1 - beta1) * dw
                    db = beta1 * self.velocities_b[i] + (1 - beta1) * db
                self.weights[i] -= learning_rate * dw / (np.sqrt(self.squared_w[i]) + epsilon)
                self.biases[i] -= learning_rate * db / (np.sqrt(self.squared_b[i]) + epsilon)

In [4]:
sweep_config = {
    "name": "Feedforward Network - Hyper parameter search",
    "metric": {
        "name": "Validation Loss",
        "goal": "minimize"
    },
    "method": "random",
    "parameters": {
        "num_epochs": {
            "values": [5, 10]
            },
        "num_hiddenLayers": {
            "values": [3, 4, 5]
            },
        "hiddenLayer_Size": {
            "values": [32, 64, 128]
            },
        "weightDecay": {
            "values": [0, 0.0005, 0.5]
            },
        "learningRate": {
            "values": [1e-3, 1e-4]
            },
        "optimizer": {
            "values": ["sgd", "momentum", "nesterov", "rmsprop", "adam", "nadam"]
            },
        "batchSize": {
            "values": [16, 32, 64]
            },
        "weightInit": {
            "values": ["random", "xavier"]
            },
        "activationFunc": {
            "values": ["tanh", "relu", "sigmoid"]
            }
    }
    
}

In [5]:
def sweep_hyperParameters():
    default_config = {
        'num_epochs': 10,
        'num_hiddenLayers': 3,
        'hiddenLayer_Size': 40,
        'weightDecay': 0,
        'learningRate': 1e-3,
        'optimizer': 'Nesterov_Accelerated_GD',
        'batchSize': 32,
        'weightInit': 'xavier',
        'activationFunc': 'sigmoid'
    }
    
    wandb.init(project="assignment1", entity="da6401-assignments")
    wandb.init(config = default_config)

    config = wandb.config # To get the Hyper Parameters from sweep_config
    
    # Parameters from wandb.config
    num_epochs = config.num_epochs
    num_hiddenLayers = config.num_hiddenLayers
    hiddenLayer_size = config.hiddenLayer_Size
    weightDecay = config.weightDecay
    learningRate = config.learningRate
    optimizer = config.optimizer
    batchSize = config.batchSize
    weightInit = config.weightInit
    activationFunc = config.activationFunc
    
    # Name
    run_name = f"LR: {learningRate}, AC: {activationFunc}, BS: {batchSize}, Optim: {optimizer}, WI: {weightInit}, WD: {weightDecay}, No_HL: {num_hiddenLayers}, HS: {hiddenLayer_size}"
    print("Sweep Name: ",run_name)
    
    model = FeedforwardNeuralNetwork(x_train.shape[0], [hiddenLayer_size] * num_hiddenLayers, y_train.shape[0])
    
    wandb.run.name = run_name
    
    model.train(x_train, y_train, x_val, y_val, num_epochs, learningRate, optimizer, batchSize)

    wandb.run.save()

    return model 

In [6]:
sweep_ID = wandb.sweep(sweep_config, entity="da6401-assignments", project="assignment1")
wandb.agent(sweep_ID, sweep_hyperParameters, count=40)

wandb.finish()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Create sweep with ID: syjentp9
Sweep URL: https://wandb.ai/da6401-assignments/assignment1/sweeps/syjentp9


wandb: Agent Starting Run: c9c39fpb with config:
wandb: 	activationFunc: tanh
wandb: 	batchSize: 64
wandb: 	hiddenLayer_Size: 32
wandb: 	learningRate: 0.001
wandb: 	num_epochs: 10
wandb: 	num_hiddenLayers: 3
wandb: 	optimizer: sgd
wandb: 	weightDecay: 0
wandb: 	weightInit: random
wandb: Currently logged in as: ed20b071 (da6401-assignments) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Sweep Name:  LR: 0.001, AC: tanh, BS: 64, Optim: sgd, WI: random, WD: 0, No_HL: 3, HS: 32
Epoch 0: Training Loss = 2.7349 Validation Loss = 2.7197
Training Accuracy = 0.0993 Validation Accuracy = 0.0958
Epoch 1: Training Loss = 2.4299 Validation Loss = 2.4256
Training Accuracy = 0.1357 Validation Accuracy = 0.1335
Epoch 2: Training Loss = 2.2886 Validation Loss = 2.2904
Training Accuracy = 0.1751 Validation Accuracy = 0.1685
Epoch 3: Training Loss = 2.1954 Validation Loss = 2.1992
Training Accuracy = 0.2210 Validation Accuracy = 0.2100
Epoch 4: Training Loss = 2.1174 Validation Loss = 2.1217
Training Accuracy = 0.2547 Validation Accuracy = 0.2483
Epoch 5: Training Loss = 2.0489 Validation Loss = 2.0531
Training Accuracy = 0.2817 Validation Accuracy = 0.2730
Epoch 6: Training Loss = 1.9881 Validation Loss = 1.9920
Training Accuracy = 0.3058 Validation Accuracy = 0.2980
Epoch 7: Training Loss = 1.9337 Validation Loss = 1.9371
Training Accuracy = 0.3250 Validation Accuracy = 0.3152
Epoch 

wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


...

Sweep Name:  LR: 0.001, AC: tanh, BS: 16, Optim: rmsprop, WI: random, WD: 0.0005, No_HL: 5, HS: 64
Epoch 0: Training Loss = 0.4940 Validation Loss = 0.5103
Training Accuracy = 0.8182 Validation Accuracy = 0.8155
Epoch 1: Training Loss = 0.4421 Validation Loss = 0.4704
Training Accuracy = 0.8376 Validation Accuracy = 0.8342
Epoch 2: Training Loss = 0.4190 Validation Loss = 0.4570
Training Accuracy = 0.8474 Validation Accuracy = 0.8362
Epoch 3: Training Loss = 0.3786 Validation Loss = 0.4265
Training Accuracy = 0.8606 Validation Accuracy = 0.8442
Epoch 4: Training Loss = 0.3608 Validation Loss = 0.4140
Training Accuracy = 0.8669 Validation Accuracy = 0.8470
Epoch 5: Training Loss = 0.3384 Validation Loss = 0.3952
Training Accuracy = 0.8758 Validation Accuracy = 0.8533
Epoch 6: Training Loss = 0.3417 Validation Loss = 0.4124
Training Accuracy = 0.8724 Validation Accuracy = 0.8563
Epoch 7: Training Loss = 0.3150 Validation Loss = 0.3881
Training Accuracy = 0.8848 Validation Accuracy = 0.86

Validation Loss,▁
epoch,▁▂▃▃▄▅▆▆▇█
train_acc,▁▃▄▅▆▇▆▇▇█
train_loss,█▆▅▄▃▃▃▂▂▁
val_acc,▁▄▄▅▆▇▇███
val_loss,█▆▅▃▂▁▂▁▁▁
Validation Loss,0.38867
epoch,9
train_acc,0.89144
train_loss,0.294
val_acc,0.8615


wandb: Agent Starting Run: y6vh5n4l with config:
wandb: 	activationFunc: sigmoid
wandb: 	batchSize: 16
wandb: 	hiddenLayer_Size: 64
wandb: 	learningRate: 0.0001
wandb: 	num_epochs: 5
wandb: 	num_hiddenLayers: 4
wandb: 	optimizer: momentum
wandb: 	weightDecay: 0.5
wandb: 	weightInit: xavier


Sweep Name:  LR: 0.0001, AC: sigmoid, BS: 16, Optim: momentum, WI: xavier, WD: 0.5, No_HL: 4, HS: 64
Epoch 0: Training Loss = 1.8045 Validation Loss = 1.7866
Training Accuracy = 0.3669 Validation Accuracy = 0.3760
Epoch 1: Training Loss = 1.5513 Validation Loss = 1.5305
Training Accuracy = 0.4625 Validation Accuracy = 0.4737
Epoch 2: Training Loss = 1.3880 Validation Loss = 1.3683
Training Accuracy = 0.5245 Validation Accuracy = 0.5397
Epoch 3: Training Loss = 1.2731 Validation Loss = 1.2575
Training Accuracy = 0.5677 Validation Accuracy = 0.5797
Epoch 4: Training Loss = 1.1883 Validation Loss = 1.1763
Training Accuracy = 0.5970 Validation Accuracy = 0.6098


Validation Loss,▁
epoch,▁▃▅▆█
train_acc,▁▄▆▇█
train_loss,█▅▃▂▁
val_acc,▁▄▆▇█
val_loss,█▅▃▂▁
Validation Loss,1.17631
epoch,4
train_acc,0.59696
train_loss,1.18829
val_acc,0.60983
